In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
import torchvision.transforms.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pydicom as dicom
import matplotlib.pyplot as plt
from skimage.io import imread
import os
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path
from torchvision import transforms as tfs
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from matplotlib import colors, pyplot as plt
%matplotlib inline


import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
from collections import defaultdict

US1_J2KR.dcm:   0%|          | 38.0/154k [00:00<01:55, 1.33kB/s]
MR-SIEMENS-DICOM-WithOverlays.dcm:   0%|          | 125/511k [00:00<03:02, 2.81kB/s]
OBXXXX1A.dcm:   0%|          | 119/486k [00:00<03:01, 2.68kB/s]
US1_UNCR.dcm:   0%|          | 226/923k [00:00<03:50, 4.01kB/s]
color3d_jpeg_baseline.dcm:   0%|          | 1.50k/6.14M [00:00<06:34, 15.6kB/s]
/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
import sys
sys.path.append('/kaggle/input/medicalnet/MedicalNet/MedicalNet')

In [10]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 69.9 MB/s eta 0:00:00:00:0100:01


In [11]:
import numpy as np
import pandas as pd

import random
from glob import glob
import os, shutil
from tqdm import tqdm
tqdm.pandas()
import time
import copy
import joblib
from collections import defaultdict
import gc
from IPython import display as ipd

# visualization
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# Sklearn
from sklearn.model_selection import StratifiedKFold, KFold, StratifiedGroupKFold

# PyTorch 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

import rasterio
from joblib import Parallel, delayed

# For colored terminal text
from colorama import Fore, Back, Style
c_  = Fore.GREEN
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import pydicom as dicom
import nibabel as nib
import sys
import glob
import os
import numpy as np
import h5py
from sklearn.preprocessing import StandardScaler

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

import timm
import tifffile as tiff

In [13]:
def read_labels_from_file(file_path):
    values = []
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            try:
                value = int(line.split(': ')[1])
            except ValueError:
                value = line.split(': ')[1].strip()  # Если значение не может быть преобразовано в int, оставляем его как строку
            if str(value) in ['3а'  ,'3','3a']:
                value = '3a'
            values.append(value)
    return values

In [14]:
def load_tif_mask(x_path, y_path, mask_pth, num_imgs=10,img_size=512):
    '''
    Перобразует tif в набор срезов с окном num_imgs на выходе  [num_imgs,img_size,img_size]

    x_path - путь до снимка 
    y_path - путь до меток 
    num_imgs - окно для взятия снимка
    img_size - размер итоговй снимка 
    '''
    images = []
    labels = []
    image =  tiff.imread(x_path)
    # mask = Image.open(mask_pth)
    i = 0
    # image = [np.fliplr(np.array(img)) for img in image]
    middle = len(image)//2
    num_imgs2 = num_imgs//2

    p1 = max(0, middle - num_imgs2)
    p2 = min(len(image), middle + num_imgs2)

    images = np.array(image[p1:p2])

    # while len(images)<10:
        # try:
        #     mask.seek(i)
        #     mask_array = np.array(mask)
        #     image.seek(i)
        #     image_array = np.array(image)
        #     if sum(sum(mask_array)) > 0:
        #         images.append(image_array)
        #         labels.append(read_labels_from_file(y_path))
        #     i += 1
        # except EOFError:
        #     break

    if len(images)<num_imgs:
        
        n_zero = np.zeros((num_imgs - images.shape[0],img_size, img_size))
        images = np.concatenate((images,  n_zero), axis =0)

    labels = read_labels_from_file(y_path)
    return images, labels

In [15]:
def load_tif_mask_inv(x_path, y_path, mask_pth, num_imgs=10,img_size=512):
    '''
    Перобразует tif в инвертирвоанный набор срезов с окном num_imgs на выходе  [num_imgs,img_size,img_size]

    x_path - путь до снимка 
    y_path - путь до меток 
    num_imgs - окно для взятия снимка
    img_size - размер итоговй снимка 
    '''
    images = []
    labels = []
    image =  tiff.imread(x_path)
    image = [np.fliplr(np.array(img)) for img in image]
    # mask = Image.open(mask_pth)
    i = 0

    middle = len(image)//2
    num_imgs2 = num_imgs//2

    p1 = max(0, middle - num_imgs2)
    p2 = min(len(image), middle + num_imgs2)

    images = np.array(image[p1:p2])


    if len(images)<num_imgs:
        
        n_zero = np.zeros((num_imgs - images.shape[0],img_size, img_size))
        images = np.concatenate((images,  n_zero), axis =0)

            
    labels = read_labels_from_file(y_path)[::-1]

    return images, labels

In [16]:
def get_pathes_mask(path):
    '''
    Возвращает массивы с путясми снимков и метками класса ( для снимков где есть маска)

    path - путь до директории со снимками 
    '''
    x_pathes_all = []
    y_pathes_all = []
    for patient in os.listdir(path):
            x_pathes = []
            y_pathes = []
        
            for ID_s in os.listdir(path + '/'+ patient ):
                if 'ID' in ID_s:
                    msk_t1 = 0
                    msk_t2 = 0  
                    for tif_name in os.listdir(path + '/'+ patient + '/'+ID_s):
                        if 'Cor' in tif_name:
                            if 'T1' in tif_name:
                
                                if  'mask' not in tif_name.lower():
                                    x_pathes.append(path + '/'+ patient + '/' + ID_s + '/'+ tif_name)

                                elif 'mask' in tif_name.lower():
                                    msk_t1 = 1
                                    y_pathes.append([path + '/'+ patient + '/'+ID_s + '/' + tif_name,path + '/'+ patient + '/' +'labels.txt'])

                            elif 'T2' in tif_name:
                                
                                if 'mask' not in tif_name.lower():
                                    x_pathes.append(path + '/'+ patient + '/'+ID_s + '/' + tif_name)
                                
                                elif 'mask' in tif_name.lower():
                                    msk_t2 = 1
                                    y_pathes.append([path + '/'+ patient + '/'+ID_s + '/' + tif_name,path + '/'+ patient + '/' +'labels.txt'])
                            
                    if msk_t1==0:
                        x_pathes.pop()
                    if msk_t2==0:
                        x_pathes.pop()
                            
            x_pathes_all.append(x_pathes)
            y_pathes_all.append(y_pathes)
    return x_pathes_all, y_pathes_all

In [17]:
def get_pathes(path):
    '''
    Возвращает массивы с путясми снимков и метками класса 

    path - путь до директории со снимками 
    '''
    x_pathes_all = []
    y_pathes_all = []
    for patient in os.listdir(path):
        x_pathes = []
        y_pathes = []
        
        for ID_s in os.listdir(path + '/'+ patient ):
            if 'ID' in ID_s:
                for tif_name in os.listdir(path + '/'+ patient + '/'+ID_s):
    
                    if 'Cor' in tif_name:
                        if 'T1' in tif_name:

                            if 'mask' not in tif_name.lower():
                                x_pathes.append(path + '/'+ patient + '/' + ID_s + '/'+ tif_name)
                                y_pathes.append(path + '/'+ patient + '/' +'labels.txt')
                            # elif 'mask' in tif_name.lower():
                            # y_pathes.append(path + '/'+ patient + '/' + tif_name)

                        elif 'T2' in tif_name:
                            
                            if 'mask' not in tif_name.lower():
                                x_pathes.append(path + '/'+ patient + '/'+ID_s + '/' + tif_name)
                                y_pathes.append(path + '/'+ patient + '/' +'labels.txt')
                            # elif 'mask' in tif_name.lower():
                            #     y_pathes.append(path + '/'+ patient + '/' + tif_name)
                    
                        
        x_pathes_all.append(x_pathes)
        y_pathes_all.append(y_pathes)
    
    return x_pathes_all, y_pathes_all

In [18]:
file_path = os.path.join('/kaggle/input/brain-tumor-mri/data_29_05_24_cls108_seg72/ID_65', 'labels.txt')

values = read_labels_from_file('/kaggle/input/brain-tumor-mri/data_29_05_24_cls108_seg72/ID_89/labels.txt')[::-1]
print(values)

['3a', 4]


In [19]:
x_pth ,y_pth = get_pathes('/kaggle/input/brain-tumor-mri/data_29_05_24_cls108_seg72')

In [20]:
def flatten(xss):# рвзвертывает список спсиков в список
    return [x for xs in xss for x in xs]

In [21]:
import torch
import numpy as np
from torch.utils.data import Dataset

class MedicalDataset(torch.utils.data.Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform
        self.knsop_labels = {'0': 0, '1': 1, '2': 2, '3a': 3, '3b': 3, '4': 4}

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        # Загружаем TIFF изображения и метки
        img_data, y_train = load_tif_mask(self.file_paths[idx], self.labels[idx], self.labels[idx])

        # Применение преобразований ко всем срезам (если есть)
        if self.transform:
            res_img = np.zeros((img_data.shape[0], 224, 224))  # Предполагаем, что выходное изображение 224x224
            for i in range(img_data.shape[0]):  # Применяем трансформации ко всем срезам
                transformed = self.transform(image=np.array(img_data[i], dtype=np.float32))
                res_img[i] = transformed['image']

            # Нормализуем данные
            res_img = (res_img - np.min(res_img)) / (np.max(res_img) - np.min(res_img))
        else:
            # Нормализация данных без применения преобразований
            res_img = (img_data - np.min(img_data)) / (np.max(img_data) - np.min(img_data))

        # Преобразуем данные к формату [1, Depth, Height, Width]
        img_data = np.expand_dims(res_img, axis=0)  # Добавляем канал (Channels = 1, Depth, Height, Width)
        
        print(f"Image shape: {img_data.shape}")  # Должен быть [1, Depth, 224, 224]
        
        Right = self.knsop_labels[str(y_train[1])]  # Используем метку для правого полушария

        sample = {
            "image": torch.from_numpy(img_data),  # Преобразуем в тензор
            "labelS": Right  # Метка
        }

        return sample



In [22]:
import torch
import numpy as np
from torch.utils.data import Dataset

class MedicalDatasetInv(torch.utils.data.Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform
        self.knsop_labels = {'0': 0, '1': 1, '2': 2, '3a': 3, '3b': 3, '4': 4}

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        # Загружаем TIFF изображения и метки
        img_data, y_train = load_tif_mask(self.file_paths[idx], self.labels[idx], self.labels[idx])

        # Применение преобразований ко всем срезам (если есть)
        if self.transform:
            res_img = np.zeros((img_data.shape[0], 224, 224))  # Предполагаем, что выходное изображение 224x224
            for i in range(img_data.shape[0]):  # Применяем трансформации ко всем срезам
                transformed = self.transform(image=np.array(img_data[i], dtype=np.float32))
                res_img[i] = transformed['image']

            # Нормализуем данные
            res_img = (res_img - np.min(res_img)) / (np.max(res_img) - np.min(res_img))
        else:
            # Нормализация данных без применения преобразований
            res_img = (img_data - np.min(img_data)) / (np.max(img_data) - np.min(img_data))

        # Преобразуем данные к формату [1, Depth, Height, Width]
        img_data = np.expand_dims(res_img, axis=0)  # Добавляем канал (Channels = 1, Depth, Height, Width)
        print(f"Image shape: {img_data.shape}")  # Должен быть [1, Depth, 224, 224]
        
        Right = self.knsop_labels[str(y_train[1])]  # Используем метку для правого полушария

        sample = {
            "image": torch.from_numpy(img_data),  # Преобразуем в тензор
            "labelS": Right  # Метка
        }

        return sample



In [23]:
data_transforms = {
    "train": A.Compose([
        A.augmentations.crops.transforms.CenterCrop(256,256),
        A.Resize(224,224, interpolation=cv2.INTER_NEAREST),
        # A.HorizontalFlip(p=0.5),
        # A.VerticalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.0225, scale_limit=0.005, rotate_limit=10, p=1.),
        A.OneOf([
            A.GridDistortion(num_steps=5, distort_limit=0.05, p=1.0),
# #             A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=1.0),
            A.ElasticTransform(alpha=1, sigma=50, alpha_affine=None, p=1.0)
        ], p=0.25),
        A.CoarseDropout(max_holes=8, max_height=224//20, max_width=224//20,
                         min_holes=5, fill_value=0, mask_fill_value=0, p=1.0),
        A.augmentations.Normalize(mean=(0.485, ), std=(0.229, )),
        ]
        , p=1.0),
    
    "valid": A.Compose([
        A.augmentations.crops.transforms.CenterCrop(256,256),
        A.Resize(224,224, interpolation=cv2.INTER_NEAREST),
        A.augmentations.Normalize(mean=(0.485, ), std=(0.229, )),
        ], p=1.0)
}

In [24]:
class CFG:
    seed          = 42
    debug         = False # set debug=False for Full Training
    exp_name      = 'Baselinev2'
    comment       = 'unet-efficientnet_b1-224x224-aug2-split2'
    model_name    = 'Eff'
    backbone      = 'efficientnet-b1'
    train_bs      = 128
    valid_bs      = train_bs*2
    img_size      = [224, 224]
    epochs        = 50
    lr            = 2e-4
    scheduler     = 'CosineAnnealingLR'
    min_lr        = 1e-6
    T_max         = int(30000/train_bs*epochs)+50
    T_0           = 25
    warmup_epochs = 0
    wd            = 1e-5
    n_accumulate  = max(1, 32//train_bs)
    n_fold        = 5
    num_classes   = 6
    device        = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [25]:
import wandb
from torchvision.models import resnet50, resnext50_32x4d,efficientnet_b6

In [26]:
def fetch_scheduler(optimizer):
    if CFG.scheduler == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CFG.T_max, 
                                                   eta_min=CFG.min_lr)
    elif CFG.scheduler == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CFG.T_0, 
                                                             eta_min=CFG.min_lr)
    elif CFG.scheduler == 'ReduceLROnPlateau':
        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer,
                                                   mode='min',
                                                   factor=0.1,
                                                   patience=7,
                                                   threshold=0.0001,
                                                   min_lr=CFG.min_lr,)
    elif CFG.scheduer == 'ExponentialLR':
        scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.85)
    elif CFG.scheduler == None:
        return None
        
    return scheduler

In [27]:
def accuracy(y_true, y_pred):
    
    """
    Function to calculate accuracy
    -> param y_true: list of true values
    -> param y_pred: list of predicted values
    -> return: accuracy score
    
    """
    correct_predictions = 0
    for yt, yp in zip(y_true, y_pred):
        if yt == yp:
            correct_predictions += 1
    return correct_predictions / len(y_true)



def recall_score_multiclass(y_true, y_pred, num_classes=6):
    recall = np.zeros(num_classes)
    for c in range(num_classes):
        true_positives = ((y_true == c) & (y_pred == c)).sum().item()
        possible_positives = (y_true == c).sum().item()
        recall[c] = true_positives / (possible_positives + 1e-7)
    return recall.mean()

def accuracy_score_multiclass(y_true, y_pred):
    correct_predictions = (y_true == y_pred).sum().item()
    accuracy = correct_predictions / y_true.numel()
    return accuracy

In [28]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(torch.tensor([1]),torch.tensor([1]), average='macro')
recall = recall_score(torch.tensor([1]),torch.tensor([1]), average='macro')
f1 = f1_score(torch.tensor([1,3]),torch.tensor([1,2]), average='macro')
print(precision,recall,f1)

1.0 1.0 0.3333333333333333


In [29]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    scaler = amp.GradScaler()
    train_scores = []
    dataset_size = 0
    running_loss = 0.0
    
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Train ')
    for step, (sample) in pbar:  
        images, label = sample['image'], sample['labelS']
 
        images  = images.to(device).half() 
        label   = label.to(device).long() 
        batch_size = images.size(0)
        
        with amp.autocast(enabled=True):

            y_pred  = model(images)

            loss   = criterion(y_pred, label)
            loss   = loss / CFG.n_accumulate
            
        scaler.scale(loss).backward()
    
        if (step + 1) % CFG.n_accumulate == 0:
            scaler.step(optimizer)
            scaler.update()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size

        y_pred = torch.argmax(y_pred, 1)
        
        precision = precision_score(label.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), average='macro')
        recall = recall_score(label.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), average='macro')
        acc = accuracy(label.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
        
        train_scores.append([precision, recall, acc])
        
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(train_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_mem=f'{mem:0.2f} GB')
        
    train_scores  = np.mean(train_scores, axis=0)    
    torch.cuda.empty_cache()
    gc.collect()
    
    return epoch_loss, train_scores

In [30]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    val_scores = []
    
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Valid ')
    for step, (sample) in pbar:  
        images, label = sample['image'], sample['labelS']
 
        images  = images.to(device).half() 
        label   = label.to(device).long() 
        
        batch_size = images.size(0)
        with amp.autocast(enabled=True):
            y_pred  = model(images)
        
            loss    = criterion(y_pred, label)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        y_pred = torch.argmax(y_pred, 1)

        precision = precision_score(label.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), average='macro')
        recall = recall_score(label.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), average='macro')
        acc = accuracy(label.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
        
        val_scores.append([precision, recall, acc])
        
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(valid_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_memory=f'{mem:0.2f} GB')
        
        
    val_scores  = np.mean(val_scores, axis=0)
    torch.cuda.empty_cache()
    gc.collect()
    
    return epoch_loss, val_scores

In [31]:
def run_training(model, optimizer, scheduler, device, num_epochs):
    # To automatically log gradients
    # wandb.watch(model, log_freq=100)
    
    if torch.cuda.is_available():
        print("cuda: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc     = -np.inf
    best_epoch     = -1
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        print(f'Epoch {epoch}/{num_epochs}', end='')
        train_loss,train_scores = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CFG.device, epoch=epoch)
        train_precision, train_recall, train_acc = train_scores

        print(f'Train Acc: {train_acc:0.4f} | Train Recall: {train_recall:0.4f}')

        val_loss, val_scores = valid_one_epoch(model, valid_loader, 
                                                 device=CFG.device, 
                                                 epoch=epoch)
        val_precision, val_recall, val_acc = val_scores
     
        history['Train Loss'].append(train_loss)
        history['Train Precision'].append(train_precision)
        history['Train Recall'].append(train_recall)
        history['Train Acc'].append(train_acc)

        history['Valid Loss'].append(val_loss)
        history['Valid Precision'].append(val_precision)
        history['Valid Recall'].append(val_recall)
        history['Valid Acc'].append(val_acc)
        
        
        # Log the metrics
        # wandb.log({"Train Loss": train_loss, 
        #            "Valid Loss": val_loss,
        #            "Valid Dice": val_dice,
        #            "Valid Jaccard": val_jaccard,
        #            "LR":scheduler.get_last_lr()[0]})
 
        print(f'Valid Acc: {val_acc:0.4f} | Valid Recall: {val_recall:0.4f}')
        
        # deep copy the model
        if val_acc > best_acc:
            print(f"{c_}Valid Score Improved ({best_acc:0.4f} ---> {val_acc:0.4f})")
            best_recall   = val_recall
            best_acc = val_acc
            best_precision = val_precision
            best_epoch   = epoch
            #run.summary["Best Recall"]    = best_recall
            #run.summary["Best Acc"] = best_acc
            #run.summary["Best Precision"] = best_precision
            #run.summary["Best Epoch"]   = best_epoch
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f"best_epoch-{fold:02d}.bin"
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            # wandb.save(PATH)
            print(f"Model Saved{sr_}")
            
        last_model_wts = copy.deepcopy(model.state_dict())
        PATH = f"last_epoch-{fold:02d}.bin"
        torch.save(model.state_dict(), PATH)
            
        print(); print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Score: {:.4f}".format(best_acc))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [32]:
class ResNMEAN(nn.Module):#Resnet 50 like feature extractor with MEAN aggregation and MLP cls
    def __init__(self,input_dim, hidden_dim, output_dim, num_layers=1, dropout=0.1):
        super(ResNMEAN, self).__init__()

        self.model = resnet50(pretrained=True)
        # for param in resnet.parameters():
        #     param.requires_grad = False
        original_conv1_weight = self.model.conv1.weight.data
        original_conv1_weight_mean = original_conv1_weight.mean(dim=1, keepdim=True)
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.model.conv1.weight.data = original_conv1_weight_mean
        
        self.model.fc = nn.Identity()
        self.fc1 = nn.Sequential(
                                nn.Linear(hidden_dim, hidden_dim//2),
                                nn.ReLU(),
                                nn.Dropout(dropout),
                                nn.Linear(hidden_dim//2, output_dim),
                                )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        batch_size, c, seq_length,  h, w = x.shape
        cnn_embed_seq = []
        for t in range(seq_length):
            # with torch.no_grad():
                x_t = self.model(x[:, :, t, :, :].to('cuda').float())  # pass each frame through ResNet
                x_t = x_t.view(batch_size, -1)
                cnn_embed_seq.append(x_t)
            
        cnn_embed_seq = torch.stack(cnn_embed_seq, dim=1)
        mean_cnn_embed = cnn_embed_seq.mean(dim=1)
        output = self.fc1(mean_cnn_embed)
        
        
        return output

In [51]:

device=CFG.device
import torch
import torch.nn as nn
from models.resnet import resnet50

class MedicalNet(nn.Module):
    def __init__(self, path_to_weights, device, num_classes=5):
        super(MedicalNet, self).__init__()
        # Инициализируем ResNet-50 с представительной глубиной
        self.model = resnet50(
            sample_input_D=16,  # Задайте представительную глубину ваших данных
            sample_input_H=224,
            sample_input_W=224,
            num_seg_classes=num_classes
        )

        # Заменяем последний слой пулинга на адаптивный, чтобы работать с разной глубиной
        self.model.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))

        # Получаем количество входных признаков для линейного слоя
        num_ftrs = self.model.conv_seg.in_features

        # Заменяем последний линейный слой на новый
        self.model.conv_seg = nn.Linear(num_ftrs, num_classes)

        # Загружаем предобученные веса
        net_dict = self.model.state_dict()
        pretrained_weights = torch.load(path_to_weights, map_location=device)
        pretrain_dict = {
            k.replace("module.", ""): v for k, v in pretrained_weights['state_dict'].items()
            if k.replace("module.", "") in net_dict.keys()
        }
        net_dict.update(pretrain_dict)
        self.model.load_state_dict(net_dict, strict=False)  # Используем strict=False для игнорирования несовпадений

    def forward(self, x):
        print(f"Input x shape: {x.shape}")  # [batch_size, channels, depth, height, width]
        features = self.model(x)
        print(f"Output features shape: {features.shape}")  # [batch_size, num_classes]
        return features


In [33]:
def flatten(xss):
    return [x for xs in xss for x in xs]

In [34]:
data_transforms = {
    "train": A.Compose([
        A.augmentations.crops.transforms.CenterCrop(256,256),
        A.Resize(224,224, interpolation=cv2.INTER_NEAREST),
        # A.HorizontalFlip(p=0.5),
        # A.VerticalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.0225, scale_limit=0.005, rotate_limit=10, p=1.),
        A.OneOf([
            A.GridDistortion(num_steps=5, distort_limit=0.05, p=1.0),
# #             A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=1.0),
            A.ElasticTransform(alpha=1, sigma=50, alpha_affine=None, p=1.0)
        ], p=0.25),
        A.CoarseDropout(max_holes=8, max_height=224//20, max_width=224//20,
                         min_holes=5, fill_value=0, mask_fill_value=0, p=1.0),
        A.augmentations.Normalize(mean=(0.485, ), std=(0.229, )),
        ]
        , p=1.0),
    
    "valid": A.Compose([
        A.augmentations.crops.transforms.CenterCrop(256,256),
        A.Resize(224,224, interpolation=cv2.INTER_NEAREST),
        A.augmentations.Normalize(mean=(0.485, ), std=(0.229, )),
        ], p=1.0)
}

In [35]:
x_pth ,y_pth = get_pathes('/kaggle/input/brain-tumor-mri/data_29_05_24_cls108_seg72')

x_pth_train,x_pth_val,y_pth_train,y_pth_val = train_test_split(x_pth , y_pth, test_size=0.2)
x_pth_train,y_pth_train,x_pth_val,y_pth_val = flatten(x_pth_train) ,flatten(y_pth_train),flatten(x_pth_val),flatten(y_pth_val)

In [36]:
!pip install GPUtil

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7394 sha256=75f61d6e02dead17693aeb1ad12881d469dc0f8f275890891b19bb96dbbcbab7
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil


In [52]:

import GPUtil


def build_model():

    input_dim = 2048  # Размерность эмбеддингов из ResNet50
    hidden_dim = 2048
    output_dim = 5  # Количество классов
    num_layers = 1
    base_model = MedicalNet(path_to_weights="/kaggle/input/medicalnet/Pretrained/Pretrained/resnet_50.pth", device=CFG.device)
    
    # Заморозка параметров
    for param_name, param in base_model.named_parameters():
        if param_name.startswith("fc1"):
            param.requires_grad = True  # Размораживаем только линейные слои
        else:
            param.requires_grad = False  # Остальные параметры заморожены

    base_model.to('cuda')
    return base_model

def criterion(y_pred, y_true):
    loss = nn.CrossEntropyLoss()
    return loss(y_pred, y_true)

def load_model(path):
    model = build_model()
    model.load_state_dict(torch.load(path))
    model.eval()
    return model

train_dataset = MedicalDataset(x_pth_train,y_pth_train,transform=data_transforms['train'])
val_dataset = MedicalDataset(x_pth_val,y_pth_val,transform=data_transforms['valid'])
train_dataset_inv = MedicalDatasetInv(x_pth_train,y_pth_train,transform=data_transforms['train'])
val_dataset_inv  = MedicalDatasetInv(x_pth_val,y_pth_val,transform=data_transforms['valid'])

train_norm_inv = torch.utils.data.ConcatDataset([train_dataset, train_dataset_inv])
val_norm_inv = torch.utils.data.ConcatDataset([val_dataset, val_dataset_inv])

train_loader = DataLoader(train_norm_inv, batch_size=4,shuffle=True)
valid_loader = DataLoader(val_norm_inv, batch_size=32, shuffle=False)
for fold in range(1):
    
    print(f'#'*15)
    print(f'### Fold: {fold}')
    print(f'#'*15)
    #run = wandb.init(project='uw-maddison-gi-tract', 
    #                config={k:v for k, v in dict(vars(CFG)).items() if '__' not in k},
    #                anonymous=anonymous,
    #                name=f"fold-{fold}|dim-{CFG.img_size[0]}x{CFG.img_size[1]}|model-{CFG.model_name}",
    #                group=CFG.comment,
    #                )
    train_loader, valid_loader = train_loader,valid_loader
    model     =   build_model()
    # unfreeze_last_n_layers(model, 2)

    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=CFG.lr, weight_decay=CFG.wd)
    scheduler =  fetch_scheduler(optimizer) 
    # fetch_scheduler(optimizer)
    
    model, history = run_training(model, optimizer, scheduler,
                                device=CFG.device,
                                num_epochs=CFG.epochs)
    # run.finish()
    # display(ipd.IFrame(run.url, width=1000, height=720))
    plt.figure(figsize=(12,9))

    plt.plot(history['Train Loss'], label=f'Training Loss')
    plt.plot(history['Valid Loss'], label=f'Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(f'Loss History-model')


    plt.figure(figsize=(17,9))
    plt.subplot(1, 2, 1)
    plt.plot(history['Train Acc'], label=f'Training Acc')
    plt.plot(history['Valid Acc'], label=f'Validation Acc')
    plt.xlabel('Epoch')
    plt.ylabel('Acc')
    plt.legend()
    plt.title(f'Acc History-model')

    plt.subplot(1, 2, 2)
    plt.plot(history['Train Recall'], label=f'Training Recall')
    plt.plot(history['Valid Recall'], label=f'Validation Recall')
    plt.xlabel('Epoch')
    plt.ylabel('Recall')
    plt.legend()
    plt.title(f'Recall History-model')
    plt.show()
    torch.save(model.state_dict(), 'lstm_bs64_state_dict')
    gpus = GPUtil.getGPUs()
    gpu = gpus[0]
    GPUtil.showUtilization()
    free_memory = gpu.memoryFree
    torch.cuda.empty_cache()
    gc.collect()

###############
### Fold: 0
###############


AttributeError: 'Sequential' object has no attribute 'in_features'

In [45]:
import GPUtil
import torch

gpus = GPUtil.getGPUs()

gpu = gpus[0]

GPUtil.showUtilization()

free_memory = gpu.memoryFree

torch.cuda.empty_cache()


| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
|  1 |  0% |  0% |


In [ ]:
import os
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
device = 'cuda'

def get_predictions(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for step, (sample) in enumerate(loader):  
            
            images, label = sample['image'], sample['labelS']
    
            images  = images.to(device).half() 
            label   = label.to(device).long() 
            print(len(label),len(images))
            batch_size = images.size(0)
            # batch_features = []
            # for img in images:
            #     # Поскольку batch_size=1
            #         img = torch.permute(img, (1, 0, 2,3))
            #         img = img.to('cuda').float()
                    
            #         features = feature_extractor(img)
            #         batch_features.append(features)
                
            #     # Получаем эмбеддинги для всех срезов
            # patient_embedding = torch.stack(batch_features).squeeze(1)
            with amp.autocast(enabled=True):
                outputs = model(images)
             
            if (len(images))==16: 
                print(outputs.size())
                print(torch.max(outputs,1))

            _,preds = torch.max(outputs,1 )
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(label.cpu().numpy())
    return all_preds, all_labels

# Получение предсказаний для тренировочного и тестового наборов
train_preds, train_labels = get_predictions(model, train_loader)
test_preds, test_labels = get_predictions(model, valid_loader)
print(len(train_preds),len(train_labels))
      # Функция для отрисовки матрицы путаницы
def plot_confusion_matrix(preds, labels,filename,title='Confusion Matrix'):
    cm = confusion_matrix(labels, preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap='Blues', xticks_rotation='vertical')
    plt.title(title)
    plt.savefig(filename)
    plt.show()

# Отрисовка матрицы путаницы для тренировочного и тестового наборов

plot_confusion_matrix(train_preds, train_labels,  title='Train Confusion Matrix',filename=os.path.join('output', 'train_confusion_matrix.png'))
plot_confusion_matrix(test_preds, test_labels, title='Test Confusion Matrix',filename=os.path.join('output', 'val_confusion_matrix.png'))